In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

Load AQI data

In [2]:
aqi_df = pd.read_csv('data/processed/cleaned/cleaned_air.csv', index_col=0, parse_dates=True)
aqi_df.index.freq = "h"

Load extra weather data

In [3]:
weather_df = pd.read_csv('data/processed/cleaned/cleaned_weather.csv', index_col=0, parse_dates=True)
weather_df.index.freq = "h"

## Evaluation Metrics

- sMAPE (Symmetric Mean Absolute Percentage Error)

- MASE (Mean Absolute Scaled Error)

In [4]:
def smape(actual, predicted):
    """Symmetric MAPE - avoids division by zero."""
    actual = np.array(actual)
    predicted = np.array(predicted)
    denominator = np.abs(actual) + np.abs(predicted)
    denominator = np.where(denominator == 0, 1, denominator)
    return np.mean(2 * np.abs(actual - predicted) / denominator)

def mase_h_step(actual, predicted, train, h):
    """MASE scaled by h-step naive error."""
    mae = mean_absolute_error(actual, predicted)
    naive_errors = np.abs(train.values[h:] - train.values[:-h])
    naive_mae = np.mean(naive_errors)
    return mae / naive_mae if naive_mae > 0 else np.nan

## Find Optimal ARIMAX Parameters (d, q)

- **p = 2**
- **d** is determined using the ADF test (stationarity check)
- **q** is found via grid search minimizing AIC

In [5]:
p = 2

def find_d(series, max_d=2):
    for d in range(max_d + 1):
        diff_series = series.diff(d).dropna() if d > 0 else series
        adf_result = adfuller(diff_series)
        if adf_result[1] < 0.05:
            return d
    return max_d

def find_best_q(series, exog, p=2, d=0, q_range=range(0, 4)):
    best_aic = float('inf')
    best_q = 0
    for q in q_range:
        try:
            model = SARIMAX(series, exog=exog, order=(p, d, q))
            fit = model.fit(disp=False)
            if fit.aic < best_aic:
                best_aic = fit.aic
                best_q = q
        except:
            continue
    return best_q, best_aic

In [6]:
aqi_comps = aqi_df.columns
best_params = {}

for comp in aqi_comps:
    print(f"Finding parameters for {comp}...")
    d = find_d(aqi_df[comp])
    q, aic = find_best_q(aqi_df[comp], weather_df, p=p, d=d)
    best_params[comp] = (p, d, q)
    print(f"  Best order: ({p}, {d}, {q}), AIC: {aic:.2f}")

print("\nOptimal parameters:")
best_params

Finding parameters for carbon_monoxide...


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood o

  Best order: (2, 0, 2), AIC: 351413.43
Finding parameters for pm10...


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood o

  Best order: (2, 0, 2), AIC: 194615.04
Finding parameters for pm2_5...


...

d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood o

  Best order: (2, 0, 0), AIC: 205757.63
Finding parameters for sulphur_dioxide...


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood o

  Best order: (2, 0, 2), AIC: 134254.37

Optimal parameters:


{'carbon_monoxide': (2, 0, 2),
 'pm10': (2, 0, 2),
 'pm2_5': (2, 0, 1),
 'nitrogen_dioxide': (2, 0, 3),
 'ozone': (2, 0, 0),
 'sulphur_dioxide': (2, 0, 2)}

## Model Cross-Validation

Evaluate at multiple forecast horizons: 1, 3, and 6 hours ahead

In [7]:
splits = 5
horizons = [1, 3, 6]
max_horizon = max(horizons)
tscv = TimeSeriesSplit(n_splits=splits, test_size=max_horizon)

In [ ]:
results_by_horizon = {h: {comp: {'smape': [], 'mase': []} for comp in aqi_comps} for h in horizons}

for comp in aqi_comps:
    p, d, q = best_params[comp]
    
    for train_index, val_index in tscv.split(aqi_df):
        train = aqi_df[comp].iloc[train_index]
        val = aqi_df[comp].iloc[val_index]
        ex_train = weather_df.iloc[train_index]
        ex_val = weather_df.iloc[val_index]
        
        model = SARIMAX(train, exog=ex_train, order=(p, d, q))
        fit = model.fit(disp=False)
        forecast = fit.forecast(steps=max_horizon, exog=ex_val)
        
        for h in horizons:
            val_h = val.iloc[:h]
            forecast_h = forecast.iloc[:h]
            
            smape_val = smape(val_h.values, forecast_h.values)
            mase_val = mase_h_step(val_h.values, forecast_h.values, train, h)
            
            results_by_horizon[h][comp]['smape'].append(smape_val)
            results_by_horizon[h][comp]['mase'].append(mase_val)

print("Cross-validation complete!")

In [9]:
for h in horizons:
    print(f"\nHorizon: {h} hour(s)")
    smape_means = {comp: np.mean(results_by_horizon[h][comp]['smape']) for comp in aqi_comps}
    mase_means = {comp: np.mean(results_by_horizon[h][comp]['mase']) for comp in aqi_comps}
    results = pd.DataFrame({'smape': smape_means, 'mase': mase_means}).T
    display(results)


Horizon: 1 hour(s)


,carbon_monoxide,pm10,pm2_5,nitrogen_dioxide,ozone,sulphur_dioxide
smape,0.097451,0.133728,0.140421,0.190009,0.641028,0.027001
mase,1.842855,0.740748,0.996249,0.706824,0.628251,0.598359



Horizon: 3 hour(s)


,carbon_monoxide,pm10,pm2_5,nitrogen_dioxide,ozone,sulphur_dioxide
smape,0.234162,0.17630,0.196110,0.305441,0.706697,0.079246
mase,1.497662,0.35514,0.477483,0.403392,0.399881,0.725203



Horizon: 6 hour(s)


,carbon_monoxide,pm10,pm2_5,nitrogen_dioxide,ozone,sulphur_dioxide
smape,0.391761,0.228857,0.263069,0.424774,0.826326,0.186269
mase,1.631774,0.445394,0.589240,0.556313,0.347941,1.006901


## Fit Model on Full Training Data

In [10]:
for comp in aqi_comps:
    p, d, q = best_params[comp]
    model = SARIMAX(aqi_df[comp], exog=weather_df, order=(p, d, q))
    fit = model.fit(disp=False)
    fit.save(f"models/arimax/{comp}.pickle")
    print(f"Saved {comp} model with order ({p}, {d}, {q})")

d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Saved carbon_monoxide model with order (2, 0, 2)


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Saved pm10 model with order (2, 0, 2)


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

Saved nitrogen_dioxide model with order (2, 0, 3)


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Saved ozone model with order (2, 0, 0)


d:\Study\Code\Projects\business-analytics-project\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Saved sulphur_dioxide model with order (2, 0, 2)
